In [15]:
import requests #call API
import pandas as pd #to change to dataframe
from time import sleep # prevent API server overload in case of blocking the requests
import os
os.makedirs("maps", exist_ok=True)

df = pd.read_csv("../data/sutton_crime_data.csv")

In [9]:
import folium
from folium.plugins import MarkerCluster

# Filter violent crime only
violent_df = df[df['category'] == 'violent-crime']

# Create a base map centered on Sutton
sutton_map = folium.Map(location=[51.3618, -0.1934], zoom_start=13)

# Add markers for each crime location
for _, row in violent_df.iterrows():
    if pd.notnull(row['lat']) and pd.notnull(row['lng']):
        folium.CircleMarker(
            location=[row['lat'], row['lng']],
            radius=3,
            color='red',
            fill=True,
            fill_opacity=0.6,
            popup=row['street_name']
        ).add_to(sutton_map)

# Save map to HTML
sutton_map.save('violent_crimes_sutton_map.html')
#http://127.0.0.1:5500/notebooks/violent_crimes_sutton_map.html

In [10]:
# Group by street name and count number of crimes
top_streets = df['street_name'].value_counts().head(5)

# Display the result as a DataFrame for readability
top_streets_df = top_streets.reset_index()
top_streets_df.columns = ['Street Name', 'Number of Crimes']

top_streets_df

,Street Name,Number of Crimes
0,On or near Shopping Area,153
1,On or near Supermarket,117
2,On or near High Street,104
3,On or near Petrol Station,69
4,On or near Sutton,52


In [11]:
import folium
from folium.plugins import MarkerCluster

# Create top 5 street list
top_street_names = top_streets_df['Street Name'].tolist()

# Filter data only for top 5 street
top5_df = df[df['street_name'].isin(top_street_names)]

# Refresh the map
top5_map = folium.Map(location=[51.3618, -0.1934], zoom_start=13)

# MarkerCluster (visual for the map)
marker_cluster = MarkerCluster().add_to(top5_map)

# Marker by street
for _, row in top5_df.iterrows():
    if pd.notnull(row['lat']) and pd.notnull(row['lng']):
        folium.CircleMarker(
            location=[row['lat'], row['lng']],
            radius=5,
            color='darkred',
            fill=True,
            fill_opacity=0.7,
            popup=f"{row['street_name']} ({row['category']})"
        ).add_to(marker_cluster)

#save file as html file
top5_map.save('top5_crime_streets_map.html')

In [13]:
# Filter for violent crimes only
violent_df = df[df['category'] == 'violent-crime']

# Count number of violent crimes per street
top10_violent_streets = violent_df['street_name'].value_counts().head(10)

# Convert to DataFrame for readability
top10_df = top10_violent_streets.reset_index()
top10_df.columns = ['Street Name', 'Number of Violent Crimes']

top10_df

,Street Name,Number of Violent Crimes
0,On or near High Street,35
1,On or near Petrol Station,21
2,On or near Supermarket,19
3,On or near Collingwood Road,18
4,On or near Parking Area,18
5,On or near Brunswick Road,17
6,On or near Nightclub,17
7,On or near Chaucer Gardens,15
8,On or near Orme Road,14
9,On or near Lodge Place,14


In [17]:
import folium
from folium.plugins import MarkerCluster

# Filter rows where the street is in top 10
top10_map_df = violent_df[violent_df['street_name'].isin(top10_df['Street Name'])]

# Initialize base map
top10_map = folium.Map(location=[51.3618, -0.1934], zoom_start=13)
marker_cluster = MarkerCluster().add_to(top10_map)

# Plot markers
for _, row in top10_map_df.iterrows():
    if pd.notnull(row['lat']) and pd.notnull(row['lng']):
        folium.CircleMarker(
            location=[row['lat'], row['lng']],
            radius=5,
            color='crimson',
            fill=True,
            fill_opacity=0.8,
            popup=f"{row['street_name']} (violent crime)"
        ).add_to(marker_cluster)

# Save the map
top10_map.save('../maps/top10_violent_streets_map.html')